In [29]:
import fasttext
import pandas as pd
import matplotlib as plt

In [30]:
MODEL_NAME = "fasttext_tweet_disaster_text_base" # (6090, 0.9213464696223317, 0.9213464696223317)
#MODEL_NAME = "fasttext_tweet_disaster_text_keyword" # (6090, 0.9121510673234812, 0.9121510673234812)
#MODEL_NAME = "fasttext_tweet_disaster_text_location" # (6090, 0.9116584564860427, 0.9116584564860427)
#MODEL_NAME = "fasttext_tweet_disaster_text_keyword_location" # (6090, 0.8978653530377668, 0.8978653530377668)

In [31]:
model = fasttext.load_model(f"../model/{MODEL_NAME}")
model.predict("this is awesome omg those people")

(('__label__nondisaster',), array([0.52777958]))

In [32]:
model.test(f"../data/{MODEL_NAME}.valid")

(6090, 0.9121510673234812, 0.9121510673234812)

In [33]:
submission_df = pd.DataFrame(columns=["id", "target"])
submission_df.set_index("id")

test_df = pd.read_csv("../data/test.csv")

for index, row in test_df.iterrows():
    text = row["text"].strip().replace('\n', ' ')

    prediction = model.predict(text)
    
    target = 0
    if prediction[0][0] == "__label__disaster":
        target = 1

    entry = {"id": row["id"], "target": target, "text": text, "confidence": prediction[1][0]}
    submission_df = pd.concat([submission_df, pd.DataFrame([entry])], ignore_index=True)

submission_df

,id,target,text,confidence
0,0,1,Just happened a terrible car crash,0.957635
1,2,0,"Heard about #earthquake is different cities, s...",0.582115
2,3,1,"there is a forest fire at spot pond, geese are...",0.726500
3,9,0,Apocalypse lighting. #Spokane #wildfires,0.818285
4,11,1,Typhoon Soudelor kills 28 in China and Taiwan,0.968380
...,...,...,...,...
3258,10861,0,EARTHQUAKE SAFETY LOS ANGELES ÛÒ SAFETY FASTE...,0.824930
3259,10865,0,Storm in RI worse than last hurricane. My city...,0.587110
3260,10868,1,Green Line derailment in Chicago http://t.co/U...,0.997382
3261,10874,1,MEG issues Hazardous Weather Outlook (HWO) htt...,0.956606


In [34]:
submission_df.drop("text", axis=1)
submission_df.drop("confidence", axis=1)
submission_df.to_csv(f"../data/{MODEL_NAME}_submission.csv", index=False)